### Procedure
We create two classes:

- **AIAgent** :  An AI Agent that query Gemma LLM using a custom prompt that instruct Gemma to generate and answer (from the query) by refering to the context (as well provided); the answer to the AI Agent query function is then returned.

- **RAGSystem** :  initialized with the dataset with Data Science information, with an AIAgent object. In the init function of this class, we ingest the data from the dataset in the vector database. This class have as well a query member function. In this function we first perform similarity search with the query to the vector database. Then, we call the generate function of the ai agent object. Before returning the answer, we use a predefined template to compose the overal response from the question, answer and the context retrieved.

## Packages instalation and configurations : 

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from IPython.display import display, Markdown
import pandas as pd
import re

## AI Agent class :

In [2]:
import os
# Set your Hugging Face API token
os.environ['HUGGINGFACE_HUB_TOKEN'] = 'Your Hugging Face API token'

In [3]:

class AIAgent:
    
    def __init__(self, model_name="google/gemma-2b-it", max_length=1000):
        self.max_length = max_length
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
            self.gemma_llm = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)
        except Exception as e:
            raise ValueError(f"Error loading model: {e}")

    def create_prompt(self, query, context):
        # Prompt template
        prompt = f"""
        You are an AI Agent specialized to answer questions about FSTT (faculty of science and technology in Tanger).
        Explain the concept or answer the question about FSTT.
        In order to create the answer,use the information from the context if it seems to be relevant to the question provided (Context). 
        and the context will be as a list, so you must use just the most relevent informations from the list.
        Answer with simple words.
        If needed, include also explanations.
        it's important to answer with french languge.
        Question: {query}
        Context: {context}
        Answer:
        """
        return prompt
    
    def generate(self, query, retrieved_info):
        prompt = self.create_prompt(query, retrieved_info)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        
        # Answer generation
        answer_ids = self.gemma_llm.generate(input_ids, max_new_tokens=self.max_length)
        
        # Decode and return the answer
        answer = self.tokenizer.decode(answer_ids[0], skip_special_tokens=True)
        
        return answer

### Test the AIAgent :

In [4]:
# Initialize the AI Agent
ai_agent = AIAgent()

/home/idriss/.local/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [28]:
class RAGSystem:
    """Sentence embedding based Retrieval Based Augmented generation.
       Given a ChromaDB collection, retriever finds num_retrieved_docs relevant documents."""
    
    def __init__(self, ai_agent, collection, num_retrieved_docs=2):
        self.num_docs = num_retrieved_docs
        self.collection = collection
        self.ai_agent = ai_agent
    
    def retrieve(self, query):
        # Retrieve top k similar documents to query
        results = self.collection.query(query_texts=[query], n_results=self.num_docs)
        docs = [result for result in results['documents']]
        return docs
    
    def query(self, query):
        # Generate the answer
        context_docs = self.retrieve(query)
        context_docs = context_docs[0]
        print(context_docs)
        
        answer = self.ai_agent.generate(query, context_docs)
        
        return answer

In [29]:
def extract_answer_from_text(text):
    answer_match = re.search(r'Answer:\s*(.*)', text, re.DOTALL)
    answer = answer_match.group(1).strip() if answer_match else ""

    return answer

In [30]:
import chromadb

# Configure the ChromaDB client with persistence
persist_directory = "/home/idriss/Desktop/chroma_db"
client2 = chromadb.PersistentClient(path=persist_directory)
collection = client2.get_collection(name="text_embeddings")

# Initialize the RAGSystem with the existing collection
rag_system = RAGSystem(ai_agent=ai_agent, collection=collection, num_retrieved_docs=15)

In [25]:
# Define the query
query1 = "donner moi le nom de Doyen de FSTT"
query2 = "donner moi les noms de tout les departements de fstt"
query3 = "donner moi les noms de tout les clubs de fstt"
query4 = "c'est quoi fstt?"
query5 = "dooner moi quelque information sur le departement GÉNIE INFORMATIQUE"
query6 = "Donne le nombre de départements avec les informations de chaque departement"
query7 = "donner moi des informations sur MST : Intelligence Artificielle et Sciences de Données"
query8 = "donner moi le nom de Coordinnateur de departement GÉNIE INFORMATIQUE"
query9 = "donner moi le programme de formation de master : Intelligence Artificielle et Sciences de Données"
query10 = "le nombre des departement"
query11 = "donner moi l'annee de creation de fstt"
query12 = "c'est annee de creation de fstt en brief"
query13 = "site web de FST tanger"
query14 = "c'est qoui le telephone de FSTT"
query15 = "annee creation de fstt"
query16 = "PRÉSENTATION GÉNÉRALE sur FST Tanger"
query17 = "address de fstt, localisation"
query18 = "Green Lab"
query19 = "Les Formations Initiale de FSTT"
query20 = "Les Formation Contunie de FSTT"
query21 = "Les Recherche scientifique dans FSTT"
query22 = "Les clubs des Etudiants en FST Tanger"
query23 = "c'est quoi le nom de chef de departement Genie Informatique"
query24 = "CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ AVANCÉE"
query25 = "Signification de MST"
query26 = "quels sont les diplômes de FST Tanger ?"
query27 = "l'objective de formation : Intelligence Artificielle et Sciences de Données"
query28 = "Le Coordinnateur de formation mst : Intelligence Artificielle et Sciences de Données"
query29 = ""

In [24]:
# Get the answer from the RAG system
response = rag_system.query(query27)

# display(Markdown(response))
print("-"*100)
print(extract_answer_from_text(response))

----------------------------------------------------------------------------------------------------
L'objectif de la formation "Intelligence Artificielle et Sciences de Données" est de fournir aux étudiants avec les connaissances et les compétences nécessaires pour réussir dans le domaine du traitement statistique de l'information et des techniques d'aide à la décision. 

La formation comprend trois semestres, chacun couvrant des aspects différents de l'intelligence artificielle et des sciences de données. Les étudiants seront également exposés à des techniques de traitement des données et à des méthodes de recherche.


In [21]:
response = rag_system.query(query9)
print("-"*100)
print(extract_answer_from_text(response))

----------------------------------------------------------------------------------------------------
Le programme de formation de master : Intelligence Artificielle et Sciences de Données est un programme de 3 semestres qui permet aux étudiants de acquérir des compétences en matière d'intelligence artificielle et de sciences de données. Le programme couvre les aspects clés de l'intelligence artificielle, tels que l'apprentissage automatique, la gestion des données et la fouille de données. Il couvre également les aspects des sciences de données, tels que les bases de données, les statistiques et la recherche.


In [31]:
### Get the answer from the RAG system
response = rag_system.query(query28)
print(extract_answer_from_text(response))

["'mst : intelligence artificielle et sciences de données' est un formation dans fst tanger de type : mst", "le coordinnateur de formation 'mst : intelligence artificielle et sciences de données' est coordinnateur pédagogique  : pr.ezziyyani mostafa          :  mezziyyani .ac.ma", "link de formation 'mst : intelligence artificielle et sciences de données' est : https://fstt.ac.ma/portail2023/mst-intelligence-artificielle-et-sciences-de-donnees/", "objective de formation : 'mst : intelligence artificielle et sciences de données', est cette formation d'excellence offre de solides connaissances en conception de systèmes d'intelligence artificielle et mathématiques appliquées afin de couvrir l'ensemble des problématiques de traitement et d'analyse des données massives que rencontre les entreprises. elle met l'accent sur l'articulation entre apprentissage automatique, gestion et fouille de grandes masses de données, paradigmes du big data, représentation des connaissances, le traitement des

In [27]:
response = rag_system.query(query16)
print(extract_answer_from_text(response))

La faculté des sciences et techniques de Tanger est un établissement universitaire spécialisé dans les sciences et techniques. Elle offre une variété de programmes d'études, notamment en sciences de la vie, sciences de la terre, génie chimique, physique, mathématiques, génie informatique, génie electrique et génie mécanique.


- https://www.kaggle.com/code/gpreda/rag-using-gemma-langchain-and-chromadb
- https://huggingface.co/google/gemma-2b-it
- 